In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("../")
print(os.getcwd())

c:\workspace\EagSession1\eag_multiserver_mcp


In [3]:
from dotenv import load_dotenv

load_dotenv()

telegram_token = os.getenv("TELEGRAM_TOKEN")

In [4]:
if telegram_token is None:
    raise ValueError("TELEGRAM_TOKEN is not set")

In [5]:
import certifi
os.environ["SSL_CERT_FILE"] = certifi.where()

In [6]:
import os
from telegram import Update
from telegram.ext import (
    ApplicationBuilder,
    CommandHandler,
    MessageHandler,
    filters,
    ContextTypes,
)

# Command handlers
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Hi! I'm your bot. Send any message and I'll echo it.")

async def send(update: Update, context: ContextTypes.DEFAULT_TYPE):
    message = " ".join(context.args)
    await update.message.reply_text(f"Sending: {message}")

async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_text = update.message.text
    user_id = update.message.from_user.id
    username = update.message.from_user.username
    print(f"[Message] From {username} ({user_id}): {user_text}")  # Logs the message
    await update.message.reply_text(f"You said: {user_text}")

In [7]:
# Build the application
app = ApplicationBuilder().token(telegram_token).build()
app.add_handler(CommandHandler("start", start))
app.add_handler(CommandHandler("send", send))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))

In [8]:
# Start polling in the background
await app.initialize()
await app.start()
await app.updater.start_polling()
print("Bot is running in Jupyter...")

Bot is running in Jupyter...


In [9]:
await app.updater.stop()
await app.stop()
await app.shutdown()
print("Bot stopped.")

Bot stopped.


Telegram-Scrapper

In [25]:
# Client parameters
API_ID   = 21387278
API_HASH = 'a0874c25221dffc664d5e0d55733fa63'
PHONE_NUM    = '+918101288487'

# Chat to inspect
chat_link = "https://t.me/soutrik_bot"

In [21]:
from telethon import TelegramClient
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch
from collections import OrderedDict
from os import path, stat, remove, makedirs
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# # Client parameters
# API_ID = os.getenv("TELEGRAM_API_ID")
# API_HASH = os.getenv("TELEGRAM_API_HASH")
# PHONE_NUM = os.getenv("TELEGRAM_PHONE")

if not all([API_ID, API_HASH, PHONE_NUM]):
    raise ValueError("TELEGRAM_API_ID, TELEGRAM_API_HASH, and TELEGRAM_PHONE must be set in .env file")

def tlg_connect(api_id, api_hash, phone_number):
    """Connect and Log-in/Sign-in to Telegram API. Request Sign-in code for first execution"""
    print('Trying to connect to Telegram...')
    client = TelegramClient("Session", api_id, api_hash)
    if not client.start():
        print('Could not connect to Telegram servers.')
        return None
    else:
        if not client.is_user_authorized():
            print('Session file not found. This is the first run, sending code request...')
            client.sign_in(phone_number)
            self_user = None
            while self_user is None:
                code = input('Enter the code you just received: ')
                try:
                    self_user = client.sign_in(code=code)
                except Exception as e:
                    if "password" in str(e).lower():
                        pw = input('Two step verification is enabled. Please enter your password: ')
                        self_user = client.sign_in(password=pw)
                    else:
                        raise e
                    if self_user is None:
                        return None
    print('Sign in success.')
    print()
    return client

def tlg_get_basic_info(client, chat):
    """Get basic information (id, title, name, num_users, num_messages) from a group/channel/chat"""
    chat_entity = client.get_entity(chat)
    num_members_offset = client(GetParticipantsRequest(channel=chat_entity,
        filter=ChannelParticipantsSearch(''), offset=0, limit=0, hash=0)).count
    num_members = client(GetParticipantsRequest(channel=chat_entity,
        filter=ChannelParticipantsSearch(''), offset=num_members_offset, limit=0, hash=0)).count
    msgs = client.get_messages(chat_entity, limit=1)
    basic_info = OrderedDict([
        ("id", msgs[0].chat_id),
        ("title", msgs[0].chat.title),
        ("username", msgs[0].chat.username),
        ("num_members", num_members),
        ("num_messages", msgs.total),
        ("supergroup", msgs[0].chat.megagroup)
    ])
    return basic_info

def tlg_get_all_members(client, chat):
    """Get all members information from a group/channel/chat"""
    chat_entity = client.get_entity(chat)
    i = 0
    members = []
    users = []
    num_members = client(GetParticipantsRequest(channel=chat_entity,
        filter=ChannelParticipantsSearch(''), offset=0, limit=0, hash=0)).count
    while True:
        participants_i = client(GetParticipantsRequest(channel=chat_entity,
            filter=ChannelParticipantsSearch(''), offset=i, limit=num_members, hash=0))
        if not participants_i.users:
            break
        users.extend(participants_i.users)
        i = i + len(participants_i.users)
    
    for usr in users:
        usr_last_connection = ""
        if hasattr(usr.status, "was_online"):
            usr_last_connection = "{}/{}/{} - {}:{}:{}".format(
                usr.status.was_online.day,
                usr.status.was_online.month,
                usr.status.was_online.year,
                usr.status.was_online.hour,
                usr.status.was_online.minute,
                usr.status.was_online.second
            )
        else:
            usr_last_connection = "The user does not share this information"
        
        usr_data = OrderedDict([
            ("id", usr.id),
            ("username", usr.username),
            ("first_name", usr.first_name),
            ("last_name", usr.last_name),
            ("last_connection", usr_last_connection)
        ])
        members.append(usr_data)
    return members

def tlg_get_messages(client, chat, num_msg):
    """Get messages data from a chat"""
    messages = []
    chat_entity = client.get_entity(chat)
    msgs = client.get_messages(chat_entity, limit=num_msg)
    
    for msg in reversed(msgs.data):
        msg_sender = msg.sender.first_name
        if msg.sender.last_name:
            msg_sender = "{} {}".format(msg_sender, msg.sender.last_name)
        if msg.sender.username:
            msg_sender = "{} ({})".format(msg_sender, msg.sender.username)
        
        msg_sent_date = "{}/{}/{}".format(msg.date.day, msg.date.month, msg.date.year)
        msg_sent_time = "{}:{}:{}".format(msg.date.hour, msg.date.minute, msg.date.second)
        
        msg_data = OrderedDict([
            ("id", msg.id),
            ("text", msg.message),
            ("sent_time", msg_sent_time),
            ("sent_date", msg_sent_date),
            ("sender_user", msg_sender),
            ("sender_user_id", msg.sender.id),
            ("reply_to", msg.reply_to_msg_id)
        ])
        messages.append(msg_data)
    return messages

def tlg_get_all_messages(client, chat):
    """Get all messages data from a chat"""
    messages = []
    chat_entity = client.get_entity(chat)
    num_msg = client.get_messages(chat_entity, limit=1).total
    msgs = client.get_messages(chat_entity, limit=num_msg)
    
    for msg in reversed(msgs):
        msg_sender = msg.sender.first_name
        if msg.sender.last_name:
            msg_sender = "{} {}".format(msg_sender, msg.sender.last_name)
        if msg.sender.username:
            msg_sender = "{} (@{})".format(msg_sender, msg.sender.username)
        
        msg_sent_date = "{}/{}/{}".format(msg.date.day, msg.date.month, msg.date.year)
        msg_sent_time = "{}:{}:{}".format(msg.date.hour, msg.date.minute, msg.date.second)
        
        msg_data = OrderedDict([
            ("id", msg.id),
            ("text", msg.message),
            ("sent_time", msg_sent_time),
            ("sent_date", msg_sent_date),
            ("sender_user", msg_sender),
            ("sender_user_id", msg.sender.id),
            ("reply_to", msg.reply_to_msg_id)
        ])
        messages.append(msg_data)
    return messages

def json_write(file_path, data):
    """Write data to a JSON file"""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def json_write_list(file_path, data_list):
    """Write a list of data to a JSON file"""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

In [23]:
tlg_connect(API_ID, API_HASH, PHONE_NUM)

Trying to connect to Telegram...
Sign in success.



C:\Users\40104002\AppData\Local\Temp\ipykernel_28436\3611719928.py:25: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  if not client.start():
C:\Users\40104002\AppData\Local\Temp\ipykernel_28436\3611719928.py:29: RuntimeWarning: coroutine 'UserMethods.is_user_authorized' was never awaited
  if not client.is_user_authorized():


In [27]:
tlg_get_basic_info(client, chat_link)

AttributeError: 'coroutine' object has no attribute 'count'

In [ ]:

def scrape_chat(chat_link):
    """Main function to scrape a chat"""
    print()
    # Create the client and connect
    client = tlg_connect(API_ID, API_HASH, PHONE_NUM)
    if client is None:
        return

    try:
        # Get chat basic info
        print("Getting chat basic info...")
        chat_info = tlg_get_basic_info(client, chat_link)

        # Create output directory and file names
        if chat_info["username"]:
            files_name = chat_info["username"]
        else:
            files_name = str(chat_info["id"])
        
        output_dir = f"./output/{files_name}"
        fjson_chat = f"{output_dir}/chat.json"
        fjson_users = f"{output_dir}/users.json"
        fjson_messages = f"{output_dir}/messages.json"

        # Save chat basic info
        if path.exists(fjson_chat):
            remove(fjson_chat)
        json_write(fjson_chat, chat_info)

        # Get and save users data
        print("Getting chat members (users) info...")
        members = tlg_get_all_members(client, chat_link)
        if path.exists(fjson_users):
            remove(fjson_users)
        json_write_list(fjson_users, members)

        # Get and save messages data
        print("Getting chat messages info...")
        messages = tlg_get_all_messages(client, chat_link)
        if path.exists(fjson_messages):
            remove(fjson_messages)
        json_write_list(fjson_messages, messages)

        print("Process completed")
        print()

    finally:
        client.disconnect()

######################### END OF TELEGRAM SCRAPPER #########################